<a href="https://colab.research.google.com/github/BelfastHowe/phase/blob/main/%E2%80%9Cpix2pix%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 11 07:04:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
! unzip phase4.zip

Archive:  phase4.zip
   creating: phase4/
   creating: phase4/train/
  inflating: phase4/train/0000.png   
  inflating: phase4/train/0001.png   
  inflating: phase4/train/0002.png   
  inflating: phase4/train/0003.png   
  inflating: phase4/train/0004.png   
  inflating: phase4/train/0005.png   
  inflating: phase4/train/0006.png   
  inflating: phase4/train/0007.png   
  inflating: phase4/train/0008.png   
  inflating: phase4/train/0009.png   
  inflating: phase4/train/0010.png   
  inflating: phase4/train/0011.png   
  inflating: phase4/train/0012.png   
  inflating: phase4/train/0013.png   
  inflating: phase4/train/0014.png   
  inflating: phase4/train/0015.png   
  inflating: phase4/train/0016.png   
  inflating: phase4/train/0017.png   
  inflating: phase4/train/0018.png   
  inflating: phase4/train/0019.png   
  inflating: phase4/train/0020.png   
  inflating: phase4/train/0021.png   
  inflating: phase4/train/0022.png   
  inflating: phase4/train/0023.png   
  inflating: phase4

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [11]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [12]:
!pip install -r requirements.txt

     |████████████████████████████████| 676 kB 11.1 MB/s 
     |████████████████████████████████| 1.8 MB 48.0 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 181 kB 68.6 MB/s 
     |████████████████████████████████| 145 kB 69.5 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=dabedf9b071d06db8d38e9398d85f92a98b753562621571fe615fb1c01555777
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=788d04c3ac277cfe51237e86e108618e4c25f70bda45d790c2b8ee397856d9fc
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5709 sha256=322d25c942d932ab36fe876929f5a1d52ccd0f0e671d

# Datasets

Download one of the official datasets with:

-   `bash ./datasets/download_pix2pix_dataset.sh [cityscapes, night2day, edges2handbags, edges2shoes, facades, maps]`

Or use your own dataset by creating the appropriate folders and adding in the images. Follow the instructions [here](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/datasets.md#pix2pix-datasets).

In [13]:
!bash ./datasets/download_pix2pix_dataset.sh facades

Specified [facades]
for details.

--2022-05-11 07:06:57--  http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30168306 (29M) [application/x-gzip]
Saving to: ‘./datasets/facades.tar.gz’

./datasets/facades. 100%[===================>]  28.77M  3.14MB/s    in 9.7s    

2022-05-11 07:07:07 (2.96 MB/s) - ‘./datasets/facades.tar.gz’ saved [30168306/30168306]

facades/
facades/test/
facades/test/27.jpg
facades/test/5.jpg
facades/test/72.jpg
facades/test/1.jpg
facades/test/10.jpg
facades/test/100.jpg
facades/test/101.jpg
facades/test/102.jpg
facades/test/103.jpg
facades/test/104.jpg
facades/test/105.jpg
facades/test/106.jpg
facades/test/11.jpg
facades/test/12.jpg
facades/test/13.jpg
facades/test/14.jpg
facades/test/15.jpg
facades/test/1

# Pretrained models

Download one of the official pretrained models with:

-   `bash ./scripts/download_pix2pix_model.sh [edges2shoes, sat2map, map2sat, facades_label2photo, and day2night]`

Or add your own pretrained model to `./checkpoints/{NAME}_pretrained/latest_net_G.pt`

In [14]:
!bash ./scripts/download_pix2pix_model.sh facades_label2photo

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [facades_label2photo]
for details.

--2022-05-11 07:07:19--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/facades_label2photo.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704720 (208M)
Saving to: ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’

./checkpoints/facad 100%[===================>] 207.62M  65.7MB/s    in 3.7s    

2022-05-11 07:07:23 (55.8 MB/s) - ‘./checkpoints/facades_label2photo_pretrained/latest_net_G.pth’ saved [217704720/217704720]



# Training

-   `python train.py --dataroot ./datasets/facades --name facades_pix2pix --model pix2pix --direction BtoA`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. Add `--direction BtoA` if you want to train a model to transfrom from class B to A.

In [ ]:
!python train.py --dataroot ./datasets/phase4 --name phase4_pix2pix --model pix2pix --direction BtoA --use_wandb

流式输出内容被截断，只能显示最后 5000 行内容。
(epoch: 31, iters: 2000, time: 0.279, data: 0.003) G_GAN: 4.513 G_L1: 0.892 D_real: 0.027 D_fake: 0.022 
(epoch: 31, iters: 2100, time: 0.067, data: 0.002) G_GAN: 2.025 G_L1: 2.139 D_real: 0.286 D_fake: 0.205 
(epoch: 31, iters: 2200, time: 0.067, data: 0.002) G_GAN: 6.381 G_L1: 1.156 D_real: 0.001 D_fake: 0.003 
(epoch: 31, iters: 2300, time: 0.066, data: 0.002) G_GAN: 0.887 G_L1: 2.418 D_real: 0.771 D_fake: 0.583 
(epoch: 31, iters: 2400, time: 0.182, data: 0.003) G_GAN: 5.065 G_L1: 2.706 D_real: 0.221 D_fake: 0.005 
(epoch: 31, iters: 2500, time: 0.065, data: 0.002) G_GAN: 0.842 G_L1: 3.702 D_real: 0.776 D_fake: 0.660 
(epoch: 31, iters: 2600, time: 0.065, data: 0.003) G_GAN: 4.767 G_L1: 1.223 D_real: 0.220 D_fake: 0.024 
(epoch: 31, iters: 2700, time: 0.066, data: 0.004) G_GAN: 1.087 G_L1: 2.871 D_real: 0.390 D_fake: 0.871 
(epoch: 31, iters: 2800, time: 0.169, data: 0.002) G_GAN: 2.967 G_L1: 3.505 D_real: 0.164 D_fake: 0.200 
(epoch: 31, iters: 2900, tim

# Testing

-   `python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_pix2pix`

Change the `--dataroot`, `--name`, and `--direction` to be consistent with your trained model's configuration and how you want to transform images.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> Note that we specified --direction BtoA as Facades dataset's A to B direction is photos to labels.

> If you would like to apply a pre-trained model to a collection of input images (rather than image pairs), please use --model test option. See ./scripts/test_single.sh for how to apply a model to Facade label maps (stored in the directory facades/testB).

> See a list of currently available models at ./scripts/download_pix2pix_model.sh

In [ ]:
!ls checkpoints/

In [ ]:
!python test.py --dataroot ./datasets/facades --direction BtoA --model pix2pix --name facades_label2photo_pretrained --use_wandb

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_fake_B.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_A.png')
plt.imshow(img)

In [ ]:
img = plt.imread('./results/facades_label2photo_pretrained/test_latest/images/100_real_B.png')
plt.imshow(img)